In [111]:
import pandas as pd
import datetime
import time
import re
import collections
import warnings
import sklearn
import pickle
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import KFold,train_test_split



In [112]:
# load the data
onDrive= False
if onDrive == True:
    path= "/content/drive/MyDrive/School/IDC/RS/Project/data/"
    from google.colab import drive
    drive.mount('/content/drive')
else:
    path= "data\\"
movies_MD_df = pd.read_csv(path + "movies_metadata.csv")
credits_df = pd.read_csv(path + "credits.csv")
ratings_df = pd.read_csv(path + "ratings.csv")
links_df = pd.read_csv(path + "links.csv")

C:\Users\oferinbar\AppData\Local\Temp\ipykernel_27208\758899751.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_MD_df = pd.read_csv(path + "movies_metadata.csv")


In [113]:
# drop unnecessary columns
movies_MD_df = movies_MD_df.drop(
    columns=[
        "adult",
        "belongs_to_collection",
        "homepage",
        "poster_path",
        "production_companies",
        "production_countries",
        "spoken_languages",
        "status",
        "tagline",
        "video",
        "budget",
        "revenue",
        "vote_average",
        "popularity",
    ]
, errors="ignore")
# Remove rows which have NaN values
movies_MD_df = movies_MD_df.dropna()
credits_df = credits_df.dropna()
links_df = links_df.dropna()
# change the id data type to int
movies_MD_df["id"] = movies_MD_df["id"].astype(int)
# merge on the id column
movies_creds_df = movies_MD_df.merge(credits_df, on="id")
# print how many items we have
print("Number of items: ", len(movies_creds_df))
links_df["tmdbId"] = links_df["tmdbId"].fillna(method="backfill").astype("int64")
links_df["movieId"]= links_df["movieId"].astype("int64")

Number of items:  44488


In [114]:
#  only keep the movies that have ratings
# movies_creds_df = movies_creds_df[movies_creds_df["id"].isin(ratings_df["movieId"])]
# print("Number of items: ", len(movies_creds_df))
#  merge the links_df with the movies_creds_df
movies_creds_df = movies_creds_df.merge(links_df, left_on="id", right_on="tmdbId", how="left")
print("Number of items: ", len(movies_creds_df))
movies_creds_df.head()


Number of items:  44618


,genres,id,imdb_id,original_language,original_title,overview,release_date,runtime,title,vote_count,cast,crew,movieId,imdbId,tmdbId
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,81.0,Toy Story,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1,114709,862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15,104.0,Jumanji,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",2,113497,8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22,101.0,Grumpier Old Men,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",3,113228,15602
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,127.0,Waiting to Exhale,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",4,114885,31357
4,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10,106.0,Father of the Bride Part II,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",5,113041,11862


In [115]:
# movies_MD_df.iloc[movies_MD_df.id.idxmax()]
# print(movies_MD_df[movies_MD_df.id == 110])
#  get the first movie in ratings that is not in movies_MD_df
# print(ratings_df[ratings_df.movieId == 110])
# get the movies that are in ratings but missing from movies_MD_df
# print(ratings_df[ratings_df.movieId.isin(movies_MD_df_ids - ratings_df_ids)])
# ratings_df_ids - movies_MD_df_ids
# movies_MD_df["id"].sort_values(ascending=True).head(10)
# movies_creds_df.head()

In [116]:
# data preparation inspired by https://www.kaggle.com/code/surenj/movielens-eda
df= movies_creds_df.copy(deep=True)
df["release_date"] = (pd.to_datetime(df["release_date"], yearfirst= True )  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
genreRegex = re.compile(r": '(.*?)'")
df["genres"] = df["genres"].apply(lambda x: ', '.join(genreRegex.findall(x)))
castRegex = re.compile(r"'name': '(.*?)'")
df["cast"] = df["cast"].apply(lambda x: ', '.join(castRegex.findall(x)))
directorRegex = re.compile(r"'Director', 'name': '(.*?)'")
df["director"] = df["crew"].apply(lambda x: ', '.join(directorRegex.findall(x)))
# writingRegex= re.compile(r"'writing', 'name': '(.*?)'")
# df["writing"] = df["crew"].apply(lambda x: ', '.join(writingRegex.findall(x)))
ScreenplayRegex= re.compile(r"'Screenplay', 'name': '(.*?)'")
df["Screenplay"] = df["crew"].apply(lambda x: ', '.join(ScreenplayRegex.findall(x)))
producerRegex= re.compile(r"'Producer', 'name': '(.*?)'")
df["producer"] = df["crew"].apply(lambda x: ', '.join(producerRegex.findall(x)))
genreCols= df.genres.str.get_dummies(sep=', ') #Creating dummy variables for genres
df = pd.concat([df, genreCols], axis=1)
dfCols2Keep= ['tmdbId','id', 'imdb_id', 'original_title',
       'overview', 'release_date', 'runtime', 'title', 'vote_count', 'cast',
       'crew', 'director',  'Screenplay', 'producer', 'Action',
       'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']
df.head()

,genres,id,imdb_id,original_language,original_title,overview,release_date,runtime,title,vote_count,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,"Animation, Comedy, Family",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",815011200,81.0,Toy Story,5415.0,...,0,0,0,0,0,0,0,0,0,0
1,"Adventure, Fantasy, Family",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,818985600,104.0,Jumanji,2413.0,...,0,0,0,0,0,0,0,0,0,0
2,"Romance, Comedy",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,819590400,101.0,Grumpier Old Men,92.0,...,0,0,0,0,1,0,0,0,0,0
3,"Comedy, Drama, Romance",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",819590400,127.0,Waiting to Exhale,34.0,...,0,0,0,0,1,0,0,0,0,0
4,Comedy,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,792374400,106.0,Father of the Bride Part II,173.0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
#  find number of movies in ratings_df that are missing from movies_MD_df
movies_MD_df_ids = set(df["movieId"])
ratings_df_ids = set(ratings_df["movieId"])
print(f' count of movies in ratings_df that are missing from movies_MD_df: {len(ratings_df_ids - movies_MD_df_ids)}')
print(f' count of movies in movies_MD_df that are missing from ratings_df: {len(movies_MD_df_ids - ratings_df_ids)}')
print(f' count of movies that are in both ratings_df and movies_MD_df: {len(ratings_df_ids & movies_MD_df_ids)}')

 count of movies in ratings_df that are missing from movies_MD_df: 1408
 count of movies in movies_MD_df that are missing from ratings_df: 705
 count of movies that are in both ratings_df and movies_MD_df: 43707


In [118]:
# make a colun for each director- only keep the top x directors
directorValueCounts= df["director"].dropna().value_counts()
directors = directorValueCounts.index.to_list()
print('top directors are directors with more than 5 movies')
topDirectors = [director for director in directors if directorValueCounts[director] > 5]
print(f'found {len(topDirectors)} top directors')
directorCols= pd.get_dummies(df["director"])
directorCols= directorCols.loc[::,topDirectors].copy()
print(f'shape of directorCols: {directorCols.shape}')
directorCols.head(1)

top directors are directors with more than 5 movies
found 1561 top directors
shape of directorCols: (44618, 1561)


,,John Ford,Michael Curtiz,Werner Herzog,Alfred Hitchcock,Georges Méliès,Woody Allen,Sidney Lumet,George Cukor,Charlie Chaplin,...,Michael Crichton,Sheldon Lettich,Philip Haas,Peter Masterson,Robert Fuest,Zeki Demirkubuz,Don McBrearty,Buster Keaton,Kihachi Okamoto,John D. Hancock
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
# make a colun for each cast member- only keep the top 15 cast members per movie
cast= df["cast"].apply(lambda x: x.split(", ")[:3]).to_list()
castFlatList= [item for sublist in cast for item in sublist]
# keep only the top x cast members
castValueCounts= pd.Series(castFlatList).value_counts()
print('top cast members are cast members with more than 10 movies')
topCast = castValueCounts[castValueCounts > 10].index.to_list()
print(f'found {len(topCast)} top cast members')
# topCast = pd.Series(castFlatList).value_counts()[:movieVector["actors"]].index.to_list()
castCols= df["cast"].apply(lambda x: x.split(", ")[:5] )
castCols= castCols.apply(pd.Series).stack()
#  only keep rows with top cast members
castCols= castCols[castCols.isin(topCast)]
castCols= pd.get_dummies(castCols.apply(pd.Series).stack()).sum(level=0)
print(f'shape of castCols: {castCols.shape}')
castCols.head(1)


top cast members are cast members with more than 10 movies
found 2060 top cast members
shape of castCols: (27668, 2060)


,,50 Cent,Aamir Khan,Aaron Eckhart,Aaron Kwok,Aaron Paul,Aaron Taylor-Johnson,Abbie Cornish,Abhishek Bachchan,Abigail Breslin,...,Yvette Mimieux,Yvonne De Carlo,Zac Efron,Zach Galifianakis,Zachary Scott,Zhang Ziyi,Zhou Xun,Zoe Saldana,Zooey Deschanel,Моррис Честнат
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
# make a colun for each Screenplay- only keep the top 5 screenplay per movie
Screenplay= df["Screenplay"].apply(lambda x: x.split(", ")[:5]).to_list()
ScreenplayFlatList= [item for sublist in Screenplay for item in sublist]
# keep only the top 200 screenplay
screenPLayValueCounts= pd.Series(ScreenplayFlatList).value_counts()
print('top screenplay are screenplay with more than 5 movies')
topScreenplay = screenPLayValueCounts[screenPLayValueCounts > 5].index.to_list()
ScreenplayCols= df["Screenplay"].apply(lambda x: x.split(", ")[:5] )
ScreenplayCols= ScreenplayCols.apply(pd.Series).stack()
#  only keep rows with top screenplay
ScreenplayCols= ScreenplayCols[ScreenplayCols.isin(topScreenplay)]
ScreenplayCols= pd.get_dummies(ScreenplayCols.apply(pd.Series).stack()).sum(level=0)
print(f'shape of ScreenplayCols: {ScreenplayCols.shape}')
ScreenplayCols.head(1)


top screenplay are screenplay with more than 5 movies
shape of ScreenplayCols: (34550, 760)


,,A.I. Bezzerides,Aaron Sorkin,Abbas Kiarostami,Abem Finkel,Aben Kandel,Adam Cooper,Adam McKay,Adam Sandler,Adolph Green,...,Winston Miller,Wolf Mankowitz,Wong Kar-wai,Woody Allen,Yasujirō Ozu,Yusuke Watanabe,Zdeněk Svěrák,Álex de la Iglesia,Æneas MacKenzie,Éric Rohmer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
# path2Keep= r"data2Keep\\" # path to save the data to
if onDrive == True:
    path2Keep= path.replace("/data", "") +  r"data2Keep/" # path to save the data to
else:
    path2Keep= r"data2Keep\\"
#  only keep movies that have ratings
df= df[df["movieId"].isin(ratings_df["movieId"])]
movies_df= pd.concat([df, directorCols, castCols, ScreenplayCols], axis=1).drop(columns= ['cast', 'crew', 'director', 'Screenplay', 'producer'])
print(f'shape of dfMovies: {movies_df.shape}')
stacked= movies_df.iloc[40705:40706].stack()
movies_df.to_pickle(path2Keep + "movies_df.pkl")
stacked[stacked!=0]
#  pickle the movies_df 

shape of dfMovies: (44618, 4414)


41327  genres                                       Science Fiction, Thriller
       id                                                             19010.0
       imdb_id                                                      tt0389912
       original_language                                                   fr
       original_title                                               Deep Evil
       overview             An alien microbe lands in remote Siberia in th...
       release_date                                              1088121600.0
       runtime                                                           90.0
       title                                                        Deep Evil
       vote_count                                                         6.0
       movieId                                                       167804.0
       imdbId                                                        389912.0
       tmdbId                                                   

In [129]:
# filter sparse movies, inspired by https://www.kaggle.com/code/morrisb/how-to-recommend-anything-deep-recommender
min_movie_ratings = 1000
filtered_movies = (ratings_df.movieId.value_counts()>min_movie_ratings)
filtered_movies = filtered_movies[filtered_movies].index.tolist()

# Filter sparse users
min_user_ratings = 50
filtered_users = (ratings_df.userId.value_counts()>min_user_ratings)
filtered_users = filtered_users[filtered_users].index.tolist()

# Actual filtering
ratings_df2= ratings_df[(ratings_df.movieId.isin(filtered_movies)) & (ratings_df.userId.isin(filtered_users))]
print('Shape User-Ratings unfiltered:\t{}'.format(ratings_df.shape))
print('Shape User-Ratings filtered:\t{}'.format(ratings_df2.shape))
# dump ratings_df2 to file



Shape User-Ratings unfiltered:	(26024289, 4)
Shape User-Ratings filtered:	(20251813, 4)


In [130]:
# randomly asign filtered_users to test and train sets
train_users, test_users = train_test_split(ratings_df2.userId.value_counts().index, test_size=0.2, random_state=42)
print(f'shapes of train_users, test_users: {train_users.shape, test_users.shape}')
#  dump train_users and test_users to disk
with open(path2Keep +'train_users.pkl', 'wb') as f:
    pickle.dump(train_users, f)
with open(path2Keep +'test_users.pkl', 'wb') as f:
    pickle.dump(test_users, f)
# randomly asign filtered_movies to test and train sets    
train_movies, test_movies = train_test_split(ratings_df2.movieId.value_counts().index, test_size=0.2, random_state=42)
#  dump train_movies and test_movies to disk
print(f'shapes of train_movies, test_movies: {train_movies.shape, test_movies.shape}')
with open(path2Keep +'train_movies.pkl', 'wb') as f:
    pickle.dump(train_movies, f)
with open(path2Keep +'test_movies.pkl', 'wb') as f:
    pickle.dump(test_movies, f)

shapes of train_users, test_users: ((81903,), (20476,))
shapes of train_movies, test_movies: ((2995,), (749,))


In [131]:
#  get the last rating for each user in the test set
lastX= 3
lastXRatingsIndex= ratings_df2.groupby('userId')['timestamp'].nlargest(lastX)
#  get the ratings from ratings_df2 that are in lastXRatings
lastXRatings= ratings_df2[ratings_df2.index.isin(lastXRatingsIndex.index.get_level_values(1))]
# last2Ratings.head()

In [132]:
# sort last2Ratings by userId and timestamp
lastXRatings= lastXRatings.sort_values(by=['userId', 'timestamp'])
print("last 5 ratings of user #4 : ")
print(ratings_df2[ratings_df2.userId == 4].sort_values(by='timestamp', ascending=False).head(5))
print("derived last X ratings of user #4:")
print(lastXRatings[lastXRatings.userId == 4].sort_values(by='timestamp', ascending=False))
#  get all ratings from ratings_df2 that are not in the lastXRatings
beforelastXRatings= ratings_df2[~ratings_df2.index.isin(lastXRatingsIndex.index.get_level_values(1))]
print("the items before the last X: ")
print(beforelastXRatings[beforelastXRatings.userId == 4].sort_values(by='timestamp', ascending=False).head(3))
print(f'shape of beforelastXRatings: {beforelastXRatings.shape}')
print(f'shape of lastXRatings: {lastXRatings.shape}')

last 5 ratings of user #4 : 
     userId  movieId  rating   timestamp
76        4     2338     2.0  1042674886
66        4     1422     4.0  1042674861
70        4     1644     3.0  1042674845
117       4     4238     4.0  1042674819
61        4      648     4.0  1042674800
derived last X ratings of user #4:
    userId  movieId  rating   timestamp
76       4     2338     2.0  1042674886
66       4     1422     4.0  1042674861
70       4     1644     3.0  1042674845
the items before the last X: 
     userId  movieId  rating   timestamp
117       4     4238     4.0  1042674819
61        4      648     4.0  1042674800
68        4     1597     3.0  1042674787
shape of beforelastXRatings: (19944676, 4)
shape of lastXRatings: (307137, 4)


In [133]:
#  pickle the last2Ratings and beforelast2Ratings to path2keep
with open(path2Keep +'lastXRatings.pkl', 'wb') as f:
    pickle.dump(lastXRatings, f)
with open(path2Keep +'beforelastXRatings.pkl', 'wb') as f:
    pickle.dump(beforelastXRatings, f)

In [134]:
# ratings_df2.to_csv(path2Keep + 'filtered_ratings_df.csv', index=False)